In [1]:
import torch

In [2]:
import math


dtype = torch.float
device = torch.device("cpu")

In [3]:
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
import torch.optim as optim
from torchvision import datasets, transforms

In [4]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
train_dataset = datasets.MNIST(root='./data', train=True, download=True,transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True,transform=transform)

#train_dataset=torch.utils.data.Subset(train_dataset,range(100))

train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=100, shuffle=True)

In [5]:
class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x


In [6]:
model = NN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 6

In [7]:
for epoch in range(num_epochs):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % 10 == 0:
            print(f"Epoch {epoch+1}/{num_epochs}, Batch {batch_idx}/{len(train_loader)}, Loss: {loss.item():.4f}")

Epoch 1/6, Batch 0/600, Loss: 2.3263
Epoch 1/6, Batch 10/600, Loss: 1.0349
Epoch 1/6, Batch 20/600, Loss: 0.5995
Epoch 1/6, Batch 30/600, Loss: 0.4544
Epoch 1/6, Batch 40/600, Loss: 0.4364
Epoch 1/6, Batch 50/600, Loss: 0.3829
Epoch 1/6, Batch 60/600, Loss: 0.3964
Epoch 1/6, Batch 70/600, Loss: 0.4007
Epoch 1/6, Batch 80/600, Loss: 0.4105
Epoch 1/6, Batch 90/600, Loss: 0.5332
Epoch 1/6, Batch 100/600, Loss: 0.5078
Epoch 1/6, Batch 110/600, Loss: 0.4148
Epoch 1/6, Batch 120/600, Loss: 0.2961
Epoch 1/6, Batch 130/600, Loss: 0.2452
Epoch 1/6, Batch 140/600, Loss: 0.3669
Epoch 1/6, Batch 150/600, Loss: 0.2225
Epoch 1/6, Batch 160/600, Loss: 0.2859
Epoch 1/6, Batch 170/600, Loss: 0.3219
Epoch 1/6, Batch 180/600, Loss: 0.1997
Epoch 1/6, Batch 190/600, Loss: 0.2455
Epoch 1/6, Batch 200/600, Loss: 0.1897
Epoch 1/6, Batch 210/600, Loss: 0.1384
Epoch 1/6, Batch 220/600, Loss: 0.3589
Epoch 1/6, Batch 230/600, Loss: 0.3782
Epoch 1/6, Batch 240/600, Loss: 0.2096
Epoch 1/6, Batch 250/600, Loss: 0.25

In [8]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        _, predicted = torch.max(output.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

print(f"Accuracy on test set: {100 * correct / total:.2f}%")

Accuracy on test set: 97.60%


In [9]:
torch.save(model.state_dict(), "model.pth")#deploying the model

In [16]:
from PIL import Image 

image_path = '4.png' # Replace 'image.png' with your image file
image = Image.open(image_path).convert('L') # Convert to grayscale
transform = transforms.Compose([
    transforms.Resize((28, 28)), # Resize to MNIST size
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,)) # Normalize the image
])
image = transform(image).unsqueeze(0) # Add a batch dimension

# Make predictions
with torch.no_grad():
    output = model(image)
    _, predicted = torch.max(output.data, 1)
print('Predicted digit:', predicted.item())

Predicted digit: 1


In [119]:

"""model = NN() # Replace 'mnist_model.pt' with your model file
model.load_state_dict(torch.load("model.pth"))
model.eval()

# Load and preprocess the image
image_path = '4.png' # Replace 'image.png' with your image file
image = Image.open(image_path).convert('L') # Convert to grayscale
transform = transforms.Compose([
    transforms.Resize((28, 28)), # Resize to MNIST size
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,)) # Normalize the image
])
image = transform(image).unsqueeze(0) # Add a batch dimension

# Make predictions
with torch.no_grad():
    output = model(image)
    _, predicted = torch.max(output.data, 1)
print('Predicted digit:', predicted.item())"""

'model = NN() # Replace \'mnist_model.pt\' with your model file\nmodel.load_state_dict(torch.load("model.pth"))\nmodel.eval()\n\n# Load and preprocess the image\nimage_path = \'4.png\' # Replace \'image.png\' with your image file\nimage = Image.open(image_path).convert(\'L\') # Convert to grayscale\ntransform = transforms.Compose([\n    transforms.Resize((28, 28)), # Resize to MNIST size\n    transforms.ToTensor(),\n    transforms.Normalize((0.1307,), (0.3081,)) # Normalize the image\n])\nimage = transform(image).unsqueeze(0) # Add a batch dimension\n\n# Make predictions\nwith torch.no_grad():\n    output = model(image)\n    _, predicted = torch.max(output.data, 1)\nprint(\'Predicted digit:\', predicted.item())'